# Image classification using TensorFlow

### Springboard Career Track Capstone Project 2

#### Linear Model in TensorFlow

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline 
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.pylabtools import figsize
figsize(12, 8)

#### Import data

Data acquired from https://www.cs.toronto.edu/~kriz/cifar.html

In [2]:
import tarfile
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close()

#### Create function to unpickle data

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#### Assign batch names, unpickle batches, and concatenate

In [4]:
f1 = 'cifar-10-batches-py\data_batch_1'
f2 = 'cifar-10-batches-py\data_batch_2'
f3 = 'cifar-10-batches-py\data_batch_3'
f4 = 'cifar-10-batches-py\data_batch_4'
f5 = 'cifar-10-batches-py\data_batch_5'
X_test = 'cifar-10-batches-py\\test_batch'
label_names = 'cifar-10-batches-py\\batches.meta'

In [5]:
b1 = unpickle(f1)
# b1

In [6]:
b1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [7]:
train_y1 = b1.get(b'labels')

In [8]:
b1data = b1.get(b'data')

In [9]:
b2 = unpickle(f2)

In [10]:
train_y2 = b2.get(b'labels')

In [11]:
b2data = b2.get(b'data')

In [12]:
b3 = unpickle(f3)

In [13]:
train_y3 = b3.get(b'labels')

In [14]:
b3data = b3.get(b'data')

In [15]:
b4 = unpickle(f4)

In [16]:
train_y4 = b4.get(b'labels')

In [17]:
b4data = b4.get(b'data')

In [18]:
b5 = unpickle(f5)

In [19]:
train_y5 = b5.get(b'labels')

In [20]:
b5data = b5.get(b'data')

In [21]:
X_train = np.concatenate((b1data, b2data, b3data, b4data, b5data), axis=0)

In [22]:
type(X_train)

numpy.ndarray

In [23]:
X_train.shape

(50000, 3072)

In [24]:
X_train

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ..., 
       [ 35,  40,  42, ...,  77,  66,  50],
       [189, 186, 185, ..., 169, 171, 171],
       [229, 236, 234, ..., 173, 162, 161]], dtype=uint8)

In [25]:
y_train = np.concatenate((train_y1, train_y2, train_y3, train_y4, train_y5), axis=0)

In [26]:
type(y_train)

numpy.ndarray

In [27]:
y_train = y_train.reshape(50000,1)

In [28]:
y_train

array([[6],
       [9],
       [9],
       ..., 
       [9],
       [1],
       [1]])

In [29]:
def one_hot_encode(x):
    """
    One hot encode a list of sample labels. Return a one-hot encoded vector for each label.
    : x: List of sample Labels
    : return: Numpy array of one-hot encoded labels
    """  
    from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
    encoder = LabelBinarizer()
    encoder.fit(range(10))

    return encoder.transform(x)

In [30]:
y_train = one_hot_encode(y_train)

In [31]:
y_train.shape

(50000, 10)

In [32]:
X_test = unpickle(X_test)

In [33]:
label_names = unpickle(label_names)

In [34]:
label_names

{b'label_names': [b'airplane',
  b'automobile',
  b'bird',
  b'cat',
  b'deer',
  b'dog',
  b'frog',
  b'horse',
  b'ship',
  b'truck'],
 b'num_cases_per_batch': 10000,
 b'num_vis': 3072}

 #### Example from https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/mnist_1.0_softmax.py
 https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.g110257a6da_0_598

Attempt 1 - Not convolutional

In [35]:
learning_rate = .001
training_iteration = 30
batch_size = 100
display_step = 10

Initialize placeholder variables

In [36]:
X = tf.placeholder(tf.float32, [None, 3072])
W = tf.Variable(tf.zeros([3072,10]))
b = tf.Variable(tf.zeros([10]))
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [37]:
X

<tf.Tensor 'Placeholder:0' shape=(?, 3072) dtype=float32>

Placeholder for correct answers

In [38]:
Y_ = tf.placeholder(tf.float32, [None, 10])
Y_

<tf.Tensor 'Placeholder_1:0' shape=(?, 10) dtype=float32>

Create model

In [39]:
Y = tf.nn.softmax(tf.matmul(tf.reshape(X,[-1,3072]), W) + b, 10)

Loss function

In [40]:
cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y))

% of correct answers found in batch

In [41]:
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Add summary ops to collect data

In [42]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

In [43]:
sess = tf.Session()
sess.run(init)

In [44]:
for i in range(100):
    # load batch of images and correct answers
    batch_X, batch_Y = X_train, y_train # need to know what to fill in here
    train_data = {X: batch_X, Y_: batch_Y}
    
    # train
    sess.run(train_step, feed_dict=train_data)
    
    # success? add code to print it
    a,c = sess.run([accuracy, cross_entropy], feed=train_data)
    
    # success on test data?
#     test_data={X: X_test, Y_: *test_labels*} # need to know what to fill in here
#     a,c = sess.run([accuracy, cross_entropy], feed=test_data)

InvalidArgumentError: Requires start <= limit when delta > 0: 11/1
	 [[Node: range_1 = Range[Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](range_1/start, Sub, range_1/delta)]]

Caused by op 'range_1', defined at:
  File "C:\Users\conee\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\conee\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\conee\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\conee\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\conee\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\conee\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Users\conee\Anaconda3\lib\site-pac